In [ ]:
from src.data.dataset_functions import get_dataframe_onlymods_full

modframe = get_dataframe_onlymods_full(True)

modframe.show(5)

In [ ]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)

In [ ]:
from src.features.feature_functions import get_latestpixels_from_box
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
import functools
from pyspark.sql.types import *

spark = SparkSession.builder.appName('Empty_Dataframe').getOrCreate()

#Mergen zweier DataFrames
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

#Ja, datacollect ist ineffizient ABER es gibt nicht so viele Zensuren, daher sollte das halb so wild sein!
mods = modframe.drop('pixel_color').drop('user_id').collect()

emp_RDD = spark.sparkContext.emptyRDD()

# Create an expected schema
columns = StructType([StructField('x',
                                IntegerType(), True),
                    StructField('y',
                                IntegerType(), True),
                    StructField('user_id',
                                StringType(), True),
                    StructField('t',
                                LongType(), True),
                    StructField('pixel_color',
                                StringType(), True)])


censoredData = spark.createDataFrame(data = emp_RDD,
                                    schema = columns)

#DataFrame für alle gemachten Zensuren mit den betroffenen Pixel generieren
for row in mods:
    min_x,min_y,max_x,max_y,tz = list(row)
    censoredData = unionAll([censoredData,get_latestpixels_from_box(dataFrame,min_x,min_y,max_x,max_y,tz)])

In [ ]:
censoredData.show(5)
total_censors = censoredData.count() #Anzahl der betroffenen Pixel speichern (später für Statistik wichtig!)

<div style="color:#4e31ce">

So bis hier wurde die Funktion nun für den GESAMTEN Datensatz verallgemeinert....

Jetzt muss DataFrame mit user_ids für Bots generiert werden und dann anhand eines Joins geschaut werden, welche Pixel zu Bots gehören.

Dann kann count() für das DataFrame mit den Bots angewendet werden und count() für die Ausgangstabelle mit den von der Zensur betroffenen
Pixeln, sodass das eine durch das andere geteilt werden kann um entsprechend das Verhältnis rauszubekommen!
</div>

<div style="color:#4e31ce">
Dann beginnen wir mal das DataFrame mit den user_ids für die Bots zu generieren.
</div>

<div>
botDiff = timeDiffReduced.alias('tdr1').join(timeDiffReduced.alias('tdr2'),F.col('tdr1.user_id') == F.col('tdr2.user_id'),'inner').join(timeDiffReduced.alias('tdr3'),F.col('tdr1.user_id') == F.col('tdr3.user_id'), 'inner')


botDiff = botDiff.where('tdr1.diff == tdr2.diff').where('tdr1.t2 == tdr2.t1').where('tdr3.diff == tdr1.diff').where('tdr2.t2 == tdr3.t1')
</div>

In [ ]:
#DataFrame ist schon vorhanden, siehe oben!
timeDiffFrame = dataFrame.alias('df1').join(dataFrame.alias('df2'),F.col('df1.user_id') == F.col('df2.user_id'),'inner')
timeDiffFrame = timeDiffFrame.where('df2.t > df1.t')
timeDiffFrame = timeDiffFrame.orderBy(['df1.user_id', 'df1.t'])
timeDiffFrame = timeDiffFrame.select(F.col('df1.user_id'),F.col('df1.t').alias('t1'),F.col('df2.t').alias('t2'),(F.col('df2.t') - F.col('df1.t')).alias('diff'))
#timeDiffFrame.printSchema()

In [ ]:
timeDiffReduced = timeDiffFrame.dropDuplicates(['user_id', 't1'])
timeDiffReduced = timeDiffReduced.orderBy(['user_id', 't1'])

In [ ]:
botDiff = timeDiffReduced.alias('tdr1').join(timeDiffReduced.alias('tdr2'),F.col('tdr1.user_id') == F.col('tdr2.user_id'),'inner').join(timeDiffReduced.alias('tdr3'),F.col('tdr1.user_id') == F.col('tdr3.user_id'), 'inner')
botDiff = botDiff.where('tdr1.diff == tdr2.diff').where('tdr1.t2 == tdr2.t1').where('tdr3.diff == tdr1.diff').where('tdr2.t2 == tdr3.t1')
botDiff = botDiff.select('tdr1.user_id','tdr1.diff')
botDiff = botDiff.dropDuplicates(['user_id'])
botDiff = botDiff.orderBy(['user_id'])
botDiff.printSchema()

In [ ]:
botDiff = botDiff.drop("diff")
botDiff.printSchema()

<div style="color:#4e31ce">
Das DataFrame botDiff ist dann unsere Liste an Bots.
</div>

<div style="color:#4e31ce">
Jetzt müssen die beiden DataFrames censoredData und botDiff gejoined werden übder die user_ids, sodass nur noch Bots erhalten bleiben.
</div>

In [ ]:
joinedCensors = censoredData.alias('cd').join(botDiff.alias('bd'),F.col('cd.user_id') == F.col('bd.user_id'),'inner')
joinedCensors.printSchema()